In [ ]:
import numpy as np
from scipy.stats import pearsonr
import sys
sys.path.append("/mnt/shared/Huggingface/sharedcode/Stegastamp_CR")
sys.path.append("/data/shared/Huggingface/sharedcode/Stegastamp_CR")
from utils.helpers import *
from dataset import CustomImageFolder
from torchvision import transforms
from functools import partial
import torch
from functions import *
from torchvision.datasets import ImageFolder
import csv



set_seeds(2024)
for mi in [""]:
    model_choice = "hidden" #"stega", hidden
    data_choice = "COCO" #"COCO", CelebA
    device = torch.device("cuda:0")
    data_dir = {"COCO":"/data/shared/coco2017/test2017","CelebA":"/data/shared/deepfake/CelebA-HQ/val"}
    ckp_dir = f"/data/shared/Huggingface/sharedcode/Stegastamp_Train/{mi}weights/{data_choice}/{model_choice}"
    transform_pipe = [
        transforms.Resize((128,128)),
        transforms.ToTensor(),
    ]
    normalize_ = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
    un_normalize_ = transforms.Normalize([-1,-1,-1],[2,2,2])
    if model_choice == 'hidden':
        transform_pipe.append(transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]))
    transform = transforms.Compose(transform_pipe)
    if data_choice == "CelebA":
        ds = ImageFolder(data_dir[data_choice],transform=transform)
    else:
        ds = CustomImageFolder(data_dir[data_choice],transform,num=1000)

    # functions
    get_wmimages = partial(get_wmimages,ds=ds,device=device,model_choice=model_choice)
    get_images = partial(get_images,ds=ds,device=device)
    extract_message = partial(extract_message,device=device,model_choice=model_choice)
    csv_file = f"/data/shared/Huggingface/sharedcode/Stegastamp_CR/pearson_results/{data_choice}_{model_choice}_{mi}.csv"

    # attack
    series_len = 100
    message_len = 100
    ckp_list = sorted(os.listdir(ckp_dir))
    ckp_list = ["OO","emperical","GN_0.5"]
    if f"{mi}OO" in ckp_list:
        ckp_list.remove(f"{mi}OO")
        ckp_list.insert(0,f"{mi}OO")
    # ckp_list = ["OO","GN_0.1",'GN_0.25','GN_0.5',"affine_0.01","affine_0.02","affine_0.03"]
    for image_i in range(500):
        for op in ckp_list:
            if op == "OO":
                ckp_path = os.path.join(ckp_dir,op,'epoch_499_state100.pth')
            elif op == 'miOO':
                ckp_path = os.path.join(ckp_dir,op,'epoch_99_state.pth')
            elif op == 'emperical':
                ckp_path = os.path.join(ckp_dir,op,'epoch_499_state.pth')
            else:
                ckp_path = os.path.join(ckp_dir,op,'epoch_99_state.pth')

            encoder,decoder = load_weights(ckp_path,model_choice,message_len)
            encoder.eval(),decoder.eval()
            base_message = generate_message(message_len,"1"*100,batch_size=1)

            wm_images,residual_predictions_0 = get_wmimages(text=base_message,image_i=range(image_i,image_i+1),encoder=encoder)
            residual_predictions_base = residual_predictions_0.mean(dim=0)

            # calculate text distance and residual distance series
            text_distances = np.zeros((series_len,))
            residual_distances = np.zeros((series_len,))

            for text_i in range(series_len):
                text_temp = generate_message(message_len,batch_size=1)
                # print(msg2str(text_temp))
                text_distances[text_i] = hamming_distance(msg2str(base_message),msg2str(text_temp))
                _,residual_predictions = get_wmimages(text=text_temp,image_i=range(image_i,image_i+1),encoder=encoder)
                residual_distances[text_i] = torch.norm(residual_predictions_base - residual_predictions.mean(dim=0)).item()

            correlation,_ = pearsonr(text_distances,residual_distances)
            csv_dict = {"model":op,"covariance":correlation}
            print(op,"pearson:",correlation)
        with open(csv_file, mode='a', newline='') as file:
            fieldnames = csv_dict.keys()
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            if file.tell() == 0:
                writer.writeheader()
            writer.writerow(csv_dict)



uncompatible keys: <All keys matched successfully>
OO pearson: 0.5296417749345326
uncompatible keys: <All keys matched successfully>
emperical pearson: 0.5579290535735437
uncompatible keys: <All keys matched successfully>
GN_0.5 pearson: 0.7723344548120864
uncompatible keys: <All keys matched successfully>
OO pearson: 0.5591595263943566
uncompatible keys: <All keys matched successfully>
emperical pearson: 0.45245871933019355
uncompatible keys: <All keys matched successfully>
GN_0.5 pearson: 0.8099077260131031
uncompatible keys: <All keys matched successfully>
OO pearson: 0.5213782048931317
uncompatible keys: <All keys matched successfully>
emperical pearson: 0.5010407852755249
uncompatible keys: <All keys matched successfully>
GN_0.5 pearson: 0.8288538318467695
uncompatible keys: <All keys matched successfully>
OO pearson: 0.43464668014270297
uncompatible keys: <All keys matched successfully>
emperical pearson: 0.5150965714579093
uncompatible keys: <All keys matched successfully>
GN_0.

KeyboardInterrupt: 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])